**BBM473 Database Laboratory (Fall 2018)**


Exercise 8: Optimization of Relational Algebra Expressions
==================================



First, execute the following codes below.

**Note:** You may need to install `markdown`:

    $ pip install markdown

In [2]:
%load_ext sql
%sql sqlite://

%load_ext autoreload
%autoreload 2

# To help render markdown
from IPython.core.display import display, HTML
from markdown import markdown
def render_markdown_raw(m): return display(HTML(markdown(m))) # must be last element of cell.
def render_markdown(m): return render_markdown_raw(m.toMD())
def cost_markdown(q): 
    q.reset_count()
    get_result(q) # run the counters
    return display(HTML(markdown("Total Reads: {0}\n\n".format(q.total_count()) + q.toCount(0))))

# import the relational algbera operators
from relation_algebra import Select, Project, Union, NJoin, CrossProduct, BaseRelation
from relation_algebra import get_result, compare_results

from display_tools import side_by_side

import random

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%%sql
drop table if exists R; create table R(A int, B int);
drop table if exists S; create table S(B int, C int);
drop table if exists T; create table T(C int, D int);

 * sqlite://
Done.
Done.
Done.
Done.
Done.
Done.


[]

We start by generating some sample data for relations $R$ and $S$:

In [4]:
for b in range(0,5,1):
    for a in range(0,10,2):
        %sql INSERT INTO R VALUES (:a, :b);
for b in range(0,5,1):
    for c in range(0,10,2):
        %sql INSERT INTO S VALUES (:b, :c);
for c in range(0,5,1):
    for d in range(0,10,2):
        %sql INSERT INTO T VALUES (:c, :d);

 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite:

Then, let's look at the following RA expression:

In [6]:
r = %sql SELECT * FROM R;
R = BaseRelation(r, name="R")
s = %sql SELECT * FROM S;
S = BaseRelation(s, name="S")

x = Project(["B"], NJoin(R,S))
render_markdown(x)
print(get_result(x))

 * sqlite://
Done.
 * sqlite://
Done.


TypeError: can only concatenate list (not "filter") to list

Let's look at what's going on under the hood using the `cost_markdown` function:

In [ ]:
cost_markdown(x)

Note that there are several simplifying assumptions in how we compute cost here.  For one, note that we don't assume anything about being able to cache intermediate results in any way (either in buffer or on disk).  For another one, note how is the join done... what algorithm are we using here?

Note that we can also just get the total count of reads:

In [ ]:
x.total_count()

### Task 1: Optimizing the IO Cost

Can you find a logically equivalent form that uses fewer total reads?

### Task 2: Comparing costs

Let's explore how the cost of the two expressions compare as the data size increases, and with different numbers of distinct values in the data.  Given that $R$ has $N$ values, $S$ has $M$ values, you can assume that $N=M$ to simplify, and that they will both have _the same number of unique $B$ values_.

You can do this any way you choose, but we outline one way below:

We'll start with a function for each cost that will take as input:
* The number of tuples in $R$, $N$
* **_The number of distinct $B$ values in $R$, $N_B$_**
* The number of tuples in $R\Join_B S$, $O_1$
* The number of tuples in $\Pi_B(R\Join_B S)$, $O_2$

Your function should return the total number of reads as in the `cost_markdown` function

In [ ]:
def cost_1(N, M, N_B, M_B, O_1, O_2):
    # YOUR CODE HERE
    return cost

def cost_2(N, M, N_B, M_B, O_1, O_2):
    # YOUR CODE HERE
    return cost

print cost_1(25, 25, 5, 5, 125, 5)
print cost_2(25, 25, 5, 5, 125, 5)

We can plot this as a function of $N$ (assume $N=M$ for simplicity), given 5 distinct $B$ values and a rough estimate of the intermediate output counts:

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

B = 5
nrange = range(5,100)

# Plot
plt.plot(nrange, [cost_1(n, n, B, B, n*B, B) for n in nrange])
plt.plot(nrange, [cost_2(n, n, B, B, n*B, B) for n in nrange])
plt.show()

### Task 3: Optimizing some other expressions

Can you optimize the below expressions, using the tools above?

#### Task 3.1

In [ ]:
x = Select("A", 2, Project(["A","C"], NJoin(R,S)))
render_markdown(x)
print get_result(x)
cost_markdown(x)

#### Task 3.2

In [ ]:
t = %sql SELECT * FROM T;
T = BaseRelation(t, name="T")

x = Select("C", 0, Project(["A","C"], Select("B", 0, NJoin(NJoin(R, S), T))))
render_markdown(x)
print get_result(x)
cost_markdown(x)

In [ ]:
%load_ext sql
%sql sqlite:///complaint.db
import time
%sql drop index if exists helpful_index;
%sql analyze

Let's play with the [consumer complaint database](https://catalog.data.gov/dataset/consumer-complaint-database) from data.gov

In [ ]:
%sql select count(*) from complaints;

In [ ]:
%sql select * from complaints limit 5;

The syntax to create a view is below
> create index (index_name) on (table)(attributes)

In [ ]:
%%sql drop index if exists zip_index;
create index zip_index on complaints(zip_code);
analyze

In [ ]:
%%sql
select zip_code, count(*) as c 
from complaints group by zip_code 
order by c desc limit 5;

In [ ]:
%%sql select product, count(*)
from complaints c1 where ZIP_CODE = '48382'
group by product;

Let's get down to business... Indexing complainers!

For pedagogical purposes, we'll use _likelihood_ function:
  * likelihood(X,Y) does not change X's value 
  * instead it tells the DBMS the fraction of calls on which we believe X holds.
      * 0.0 is lowest, 1.0 is highest.
  * It's a form of _hint_ for the optimizer

In [ ]:
%%sql 
explain query plan 
select product, count(*) from complaints c1 
where likelihood(ZIP_CODE = '48383' or ZIP_CODE='44444', 1.0)
group by product;

In [ ]:
%%sql 
explain query plan 
select product, count(*) from complaints c1 
where likelihood(ZIP_CODE = '48383' or ZIP_CODE='44444', 0.0)
group by product;

So... does it make a difference in execution time? 

... Well, some it's a small database in memory...

In [ ]:
start_scan = time.time()
%sql select product, count(*) from complaints c1 where likelihood(ZIP_CODE = '48382' or ZIP_CODE='44444', 1.0) group by product;
scan_time = time.time() - start_scan

start_index = time.time()
%sql select product, count(*) from complaints c1 where likelihood(ZIP_CODE = '48382' or ZIP_CODE='44444', 0.0) group by product;
index_time  = time.time() - start_index
print("time for scan={0} time for index={1}".format(scan_time, index_time))
print("\t So about {0} times faster".format(scan_time/index_time))

<center>Covering Indexes</center>

We say that an index is _covering_ if all the needed columns are in the index.
 * Needed columns is the union of those in WHERE _and_ SELECT.
 * Means that the query can be answere using the index--without reading the table!

In [ ]:
%%sql drop index if exists helpful_index;
create index helpful_index on 
complaints(zip_code, product, complaint_id);

Which plan happens for this query?
> select product, count(*)<br>
> from complaints <br>
> where likelihood(ZIP_CODE = '48382', 1.0)
> <br>group by product;

NB: Using zip_index, this would be a scan!

In [ ]:
%%sql
explain query plan select product, count(*) 
from complaints c1 
where likelihood(ZIP_CODE = '48382' or ZIP_CODE='44444', 1.0) group by product;

* The index is _smaller_ in absolute data size.
* The DBMS knows this, and it picks the right index!

Summary
-------

* This notebook walks through index creation 
* That index selection is done in a cost based way.
* Introduced the notion of covering index.
* Demonstrated that not only keys to answer query, but attributes returned important component of selection. 